In [27]:
import pandas as pd
import numpy as np
import re
import Levenshtein

df = pd.read_csv(r'D:\Users\peter\PythonProjects\PandasTest\data\Week 05 Wed.csv')

#df.info()
#df

In [28]:
# Convert the column to dtype 'datatime'
df['Submitted at'] = pd.to_datetime(df['Submitted at'])

# Convert the column to dtype 'object' to handle strings properly
df['surveyid'] = df['surveyid'].astype('object')

# Replace 'Untitled multiple choice field' with 'More Info' in all column names
df.columns = df.columns.str.replace(r'Untitled multiple choice field', 'More Info')
#df

# Remove the 'Submitted at' column...
column = df.pop('Submitted at')

# Insert 'Submitted at' column at index position 6
df.insert(6, 'Submitted at', column)

# Insert two new columns that we need later
df.insert(loc=6, column='Response_Count', value='0')
df.insert(loc=0, column='Is_Duplicate', value=False)

#df.info()
#df

In [29]:
# Drop all rows when 'can_id' is NaN

df.dropna(subset=['can_id'], inplace=True)

#df

In [30]:
# Fill 'surveyid' with the specified string
# How do we pass this string into the DataFrame script?

df['surveyid'] = df['surveyid'].fillna('Week 05 Wed')

#df

In [31]:
# Email validation pattern
email_pattern = re.compile(r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$")

# Common email domains
common_domains = {
    "350seattle.org": "350seattle.org",
    "aol.com": "aol.com",
    "comcast.net": "comcast.net",
    "duck.com": "duck.com",
    "fastmail.com": "fastmail.com",
    "frontier.com": "frontier.com",
    "gmail.com": "gmail.com",
    "hotmail.com": "hotmail.com",
    "icloud.com": "icloud.com",
    "mac.com": "mac.com",
    "me.com": "me.com",
    "msn.com": "msn.com",
    "outlook.com": "outlook.com",
    "pobox.com": "pobox.com",
    "proton.me": "proton.me",
    "protonmail.com": "protonmail.com",
    "uw.edu": "uw.edu",
    "yahoo.com": "yahoo.com"
}

# Common TLD misspellings
tld_corrections = {
    'com': ['copm', 'co', 'cpm', 'comm', 'copm'],
    'org': ['ogr', 'orgg', 'or'],
    'net': ['nett', 'ne', 'nte'],
    'gov': ['gvo', 'govv', 'go'],
    'edu': ['ed', 'edd', 'eud']
}

# Function to correct common TLD misspellings
def correct_tld(domain):
    tld = domain.split('.')[-1]
    base_domain = '.'.join(domain.split('.')[:-1])
    for correct_tld, misspellings in tld_corrections.items():
        if tld in misspellings:
            return f"{base_domain}.{correct_tld}"
    return domain

# Function to validate and correct email addresses
def validate_and_correct_email(email):
    if isinstance(email, str) and email_pattern.match(email):
        domain = email.split('@')[1]
        corrected_domain = correct_tld(domain)
        if corrected_domain in common_domains:
            return email.split('@')[0] + '@' + corrected_domain
        else:
            # Find the closest match if the domain is incorrect
            corrected_domain = min(common_domains.keys(), key=lambda k: Levenshtein.distance(corrected_domain, k))
            return email.split('@')[0] + '@' + corrected_domain
    else:
        return None

# Convert all email values to strings, replacing NaN with empty string
df['Email'] = df['Email'].astype(str).fillna('')

# Validate and correct email addresses
df['Email'] = df['Email'].apply(validate_and_correct_email)

#df.info()
#df

In [32]:
# Setup df to compare and fill empty 'Email' and 'Name' values based on 'Respondent ID'

# Identify duplicates in 'Respondent ID' with 'True' in 'Is_Duplicate' column
df['Is_Duplicate'] = df['Respondent ID'].duplicated(keep=False)

# Sort by 'Respondent ID' and reset index
sorted_df = df.sort_values(by=['Respondent ID'], ascending=[True])
sorted_df = sorted_df.reset_index(drop=True) # Reset index

df = sorted_df
#df

In [33]:
# Identify the duplicates for 'Respondent ID'
duplicate_respondents = df[df.duplicated('Respondent ID', keep=False)]

# Group by 'Respondent ID' to handle each set of duplicates
grouped = duplicate_respondents.groupby('Respondent ID')

# Function to fill empty values in the 'Email' and 'Name' columns for each group
def fill_columns(group):
    group = group.copy()  # Avoid SettingWithCopyWarning
    
    # Find non-NaN values if they exist
    non_nan_email = group['Email'].dropna().iloc[0] if not group['Email'].dropna().empty else None
    non_nan_name = group['Name'].dropna().iloc[0] if not group['Name'].dropna().empty else None
    
    # Fill NaN values with non-NaN values found within the group
    if non_nan_email:
        group['Email'] = group['Email'].fillna(non_nan_email)
    if non_nan_name:
        group['Name'] = group['Name'].fillna(non_nan_name)
    
    return group

# Apply the function to each group and reset the index
RespondentID_groups = grouped.apply(lambda x: fill_columns(x.reset_index(drop=True))).reset_index(drop=True)

# Combine the manipulated groups with the original DataFrame
# First, drop the rows that were manipulated from the original DataFrame
df_non_duplicates = df[~df['Respondent ID'].isin(RespondentID_groups['Respondent ID'])]

# Concatenate the manipulated groups with the non-duplicate rows
final_df = pd.concat([df_non_duplicates, RespondentID_groups]).sort_values(by='Respondent ID').reset_index(drop=True)

df = final_df
#df

C:\Users\peter\AppData\Local\Temp\ipykernel_6376\1744984796.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  RespondentID_groups = grouped.apply(lambda x: fill_columns(x.reset_index(drop=True))).reset_index(drop=True)


In [34]:
# Setup df to compare and fill empty 'Email' and 'Name' values based on 'can_id'

# Identify duplicates in 'can_id' with 'True' in 'Is_Duplicate' column
df['Is_Duplicate'] = df['can_id'].duplicated(keep=False)

# Sort by 'can_id' and reset index
sorted_df = df.sort_values(by=['can_id'], ascending=[True])
sorted_df = sorted_df.reset_index(drop=True) # Reset index

df = sorted_df
#df

In [35]:
# Identify the duplicates for 'can_id'
duplicate_candidates = df[df.duplicated('can_id', keep=False)]

# Group by 'can_id' to handle each set of duplicates
can_id_groups = duplicate_candidates.groupby('can_id')

# Function to fill 'Email' and 'Name' columns for each group
def fill_columns(group):
    group = group.copy()  # Avoid SettingWithCopyWarning
    
    # Find non-NaN values if they exist
    non_nan_email = group['Email'].dropna().unique()
    non_nan_name = group['Name'].dropna().unique()
    
    # Check for the unique non-NaN values for 'Email' or 'Name'
    if len(non_nan_email) == 1 and len(non_nan_name) == 1:
        group['Email'] = group['Email'].fillna(non_nan_email[0])
        group['Name'] = group['Name'].fillna(non_nan_name[0])

    elif len(non_nan_email) == 1 and len(non_nan_name) == 0:
        group['Email'] = group['Email'].fillna(non_nan_email[0])

    elif len(non_nan_email) == 0 and len(non_nan_name) == 1:
        group['Name'] = group['Name'].fillna(non_nan_name[0])
    
    return group

# Apply the function to each group and reset the index
can_id_groups_filled = can_id_groups.apply(lambda x: fill_columns(x.reset_index(drop=True))).reset_index(drop=True)

# Combine the filled groups with the original DataFrame
# First, drop the rows that were manipulated from the original DataFrame
df_non_duplicates = df[~df['can_id'].isin(can_id_groups_filled['can_id'])]

# Concatenate the filled groups with the non-duplicate rows
final_df = pd.concat([df_non_duplicates, can_id_groups_filled]).sort_values(by='can_id').reset_index(drop=True)

df = final_df
#df

C:\Users\peter\AppData\Local\Temp\ipykernel_6376\3683480017.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  can_id_groups_filled = can_id_groups.apply(lambda x: fill_columns(x.reset_index(drop=True))).reset_index(drop=True)


In [36]:
# Use fillna to replace the remaining NaN 'Email' values with 'can_id'
df['Email'] = df['Email'].fillna(df['can_id'])
#df

In [37]:
# Setup df for comparing and updating empty column values for duplicate rows

# Identify duplicates in 'can_id' with 'True' in 'Is_Duplicate' column
df['Is_Duplicate'] = df['Email'].duplicated(keep=False)

# Sort by 'Email' and 'Submitted at' in ascending order
sorted_df = df.sort_values(by=['Email', 'Submitted at'], ascending=[True, True])

sorted_df = sorted_df.reset_index(drop=True) # Reset index
df = sorted_df
#df

In [38]:
# Compare and update empty column values for duplicate rows so that we retain all action taker's entered values
# Then drop the rows that are unneeded duplicates

indices_to_drop = []

for x in range(len(df) - 1):  # Iterate until the second last row
	if (df.loc[x, 'Email'] == df.loc[x + 1, 'Email']):

		# Iterate over each column starting from the 10th column (index 9 since indexing is zero-based)
		for col in df.columns[9:]:
			if pd.isna(df.at[x+1, col]):  # Check if the next row's value is empty
				df.at[x+1, col] = df.at[x, col]  # Set the value to the current row's value
	
		indices_to_drop.append(x)
	
df = df.drop(indices_to_drop)

df = df.reset_index(drop=True) # Reset index

#df

In [39]:
# Starting in the 9th column, count the number of non-NaN values in each row and store in 'Response_Count'
# However, do NOT count values in the UX columns
# 'Response_Count' quantifies each action taker's level of engagement at the survey level
# NOTE: We should NOT include rows of data in certain analyses if 'Response_Count' == 0

# List of strings to exclude columns
exclude_strings = ['More Info', 'Additional ideas for your comment', 'Email link not work for you']

# Create a mask for columns to exclude
exclude_mask = df.columns.str.contains('|'.join(exclude_strings))

# Create a mask for columns to include (starting from the 10th column, index 9)
include_mask = ~exclude_mask & (np.arange(len(df.columns)) >= 9)

# Count non-NaN values in each row for the included columns and store in 'Response_Count'
df['Response_Count'] = df.loc[:, include_mask].notna().sum(axis=1)

#df

In [40]:

# Cleanup of unneeded column
df = df.drop('Is_Duplicate', axis=1)

# Define file path
# csv_path = r'D:\Users\peter\PythonProjects\PandasTest\data\Week 05 Wed_cleaned.csv'

# Write DataFrame to CSV file
# df.to_csv(csv_path, index=False)

In [41]:
# Determine the attribute columns starting from the 7th column (index 6)
attribute_columns = df.columns[6:]

# Ensure that we have the correct columns
# print("Attribute columns:", attribute_columns)

# Unpivot the DataFrame
normalized_df = pd.melt(df, id_vars=df.columns[:6], value_vars=attribute_columns, var_name='Attribute', value_name='Value')

# Convert 'Attribute' to a categorical type to maintain the order dynamically
normalized_df['Attribute'] = pd.Categorical(normalized_df['Attribute'], categories=attribute_columns, ordered=True)

# Sort by 'ID' and the categorical 'Attribute' to maintain the sequence and reset index
normalized_df = normalized_df.sort_values(['Email', 'Attribute']).reset_index(drop=True)

In [42]:
# Create a new 'Key ID' column by concatenating 'Submission ID' + a number in the format of 01, 02, 03, etc.

# Create the 'Key ID' column with the formatted string
normalized_df.insert(0, 'Key ID', normalized_df.groupby('Submission ID').cumcount() + 1)

# Concatenate 'Submission ID' to the 'Key ID' two-digit number with leading zeros
normalized_df['Key ID'] = normalized_df['Submission ID'] + normalized_df['Key ID'].apply(lambda x: f"{x:02}")

# Display the updated DataFrame
# print(normalized_df)


In [43]:
# Calculate 'Action Number' column

# Initialize an empty list to store the action numbers
action_numbers = []

# Iterate over each group in the DataFrame based on 'Submission ID'
for _, group in normalized_df.groupby('Submission ID'):
    # Initialize action number counter for each group
    action_number = 1
    
    # Iterate through the 'Attribute' column in the group
    for attribute in group['Attribute']:
        # Check if the current attribute contains 'Response_Count' or 'Submitted at'
        if 'Response_Count' in attribute or 'Submitted at' in attribute:
            # Append None to action_numbers if condition is met
            action_numbers.append(0)
        else:
            # Append the current action number to the list
            action_numbers.append(action_number)
            
            # Check if the current attribute marks the end of a subgroup
            if 'Run into issues with' in attribute:
                # Increment the action number for the next subgroup
                action_number += 1

# Assign the list of action numbers to the new 'Action Number' column in the DataFrame
normalized_df['Action Number'] = action_numbers

In [44]:
# Calculate 'Attribute Type' column

# Define conditions and corresponding values
conditions = [
    normalized_df['Attribute'].str.contains('Response_Count', case=False, na=False),
    normalized_df['Attribute'].str.contains('Submitted at', case=False, na=False),
    normalized_df['Attribute'].str.contains('More Info', case=False, na=False),
    normalized_df['Attribute'].str.contains('Additional ideas for your comment', case=False, na=False),
    normalized_df['Attribute'].str.contains('Email link not work for you', case=False, na=False),
    normalized_df['Attribute'].str.contains('sign in for', case=False, na=False),
    normalized_df['Attribute'].str.contains('sign in OR', case=False, na=False),
    normalized_df['Attribute'].str.contains('Did you email', case=False, na=False),
    normalized_df['Attribute'].str.contains('form email', case=False, na=False),
    normalized_df['Attribute'].str.contains('leave a comment for', case=False, na=False),
    normalized_df['Attribute'].str.contains('send a comment to', case=False, na=False),
    normalized_df['Attribute'].str.contains('issues', case=False, na=False)
]

# Define corresponding Attribute Type values
choices = [
    'Response Count',
    'Submitted Datetime',
    'More Info',
    'Action Help',
    'Action Help',
    'Action Verification',
    'Action Verification',
    'Action Verification',
    'Action Verification',
    'Action Verification',
    'Action Verification',
    'Issues'
]

# Create new column 'Attribute Type'
normalized_df['Attribute Type'] = np.select(conditions, choices, default='')

In [45]:
# Define a function to calculate the 'Action Type' based on the 'Attribute' column and number of rows
def calculate_action_type(group):
    if len(group) == 4:
        # Define the conditions for 4-row groups
        conditions_4 = [
            'sign in for' in group.iloc[2]['Attribute'] if len(group) > 2 else False,
            'sign in OR' in group.iloc[2]['Attribute'] if len(group) > 2 else False,
            'Did you email' in group.iloc[2]['Attribute'] if len(group) > 2 else False,
            'form email' in group.iloc[2]['Attribute'] if len(group) > 2 else False,
            'leave a comment for' in group.iloc[2]['Attribute'] if len(group) > 2 else False,
            'send a comment to' in group.iloc[2]['Attribute'] if len(group) > 2 else False
        ]
        
        choices_4 = [
            'Sign in', 
            'Sign in OR Written Testimony', 
            'Email', 
            'Partner Form Email', 
            'Written Testimony', 
            'Comment to District Legislator'
        ]
        
        group['Action Type'] = np.select(conditions_4, choices_4, default=np.nan)

    if len(group) == 5:
        # Define the conditions for 5-row groups
        conditions_5 = [
            'sign in for' in group.iloc[3]['Attribute'] if len(group) > 3 else False,
            'leave a comment for' in group.iloc[3]['Attribute'] if len(group) > 3 else False,
            'send a comment to' in group.iloc[3]['Attribute'] if len(group) > 3 else False
        ]
        
        choices_5 = [
            'Sign in with companion bill', 
            'Written Testimony with companion bill', 
            'Comment to District Legislator with companion bill'
        ]

        group['Action Type'] = np.select(conditions_5, choices_5, default=np.nan)

    return group

# Group by 'Submission ID' and 'Action Number', then apply the function
normalized_df = normalized_df.groupby(['Submission ID', 'Action Number'], as_index=False).apply(calculate_action_type).reset_index(drop=True)


C:\Users\peter\AppData\Local\Temp\ipykernel_6376\2605610769.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  normalized_df = normalized_df.groupby(['Submission ID', 'Action Number'], as_index=False).apply(calculate_action_type).reset_index(drop=True)


In [46]:
# Calculate the 'Action Completed' column

# Define a function to apply conditions based on the 'Value' column and number of rows
def calculate_action_completed(group):
    # Fill NaN values with empty strings to avoid TypeError
    group['Value'] = group['Value'].fillna('')
    group['Action Type'] = group['Action Type'].fillna('')

    # Check if the group has 4 rows
    if len(group) == 4:
        value_2 = group.iloc[2]['Value']
        action_type_2 = group.iloc[2]['Action Type']

        if value_2 == '':
            group['Action Completed'] = 'Skipped'
        elif 'Sign in OR Written Testimony' in action_type_2 and ('sign in for' in value_2 or 'and left a comment' in value_2):
            group['Action Completed'] = 'Yes'
        elif 'No, I did not' in value_2:
            group['Action Completed'] = 'No'
        elif 'Yes' in value_2:
            group['Action Completed'] = 'Yes'

    # Check if the group has 5 rows
    elif len(group) == 5:
        value_2 = group.iloc[2]['Value']
        value_3 = group.iloc[3]['Value']

        if value_2 == '' and value_3 == '':
            group['Action Completed'] = 'Skipped'
        elif 'Yes' in value_2 or 'Yes' in value_3:
            group['Action Completed'] = 'Yes'
        elif 'No, I did not' in value_2 and 'No, I did not' in value_3:
            group['Action Completed'] = 'No'

    return group

# Group by 'Submission ID' and 'Action Number', then apply the function
normalized_df = normalized_df.groupby(['Submission ID', 'Action Number'], as_index=False).apply(calculate_action_completed).reset_index(drop=True)


C:\Users\peter\AppData\Local\Temp\ipykernel_6376\2758427616.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  normalized_df = normalized_df.groupby(['Submission ID', 'Action Number'], as_index=False).apply(calculate_action_completed).reset_index(drop=True)


In [47]:
# Create 'Contact Type' column based on 'Attribute' column (special case when 'Action Type' = 'Sign in OR Written Testimony')

def calculate_contact_type(row):
    # Step 1: Check 'Action Verification' and 'Action Completed' values
    if row['Attribute Type'] == 'Action Verification' and row['Action Completed'] == 'Yes':

        # Step 2: Handle case for 'Action Type' = 'Sign in OR Written Testimony'
        if row['Action Type'] == 'Sign in OR Written Testimony':
            if 'and signed in' in row['Value']:
                return 'Sign in'
            elif 'and left a comment' in row['Value']:
                return 'Written Testimony'
        
        # Step 3: Other conditions based on 'Attribute' values
        elif 'sign in for' in row['Attribute']:
            return 'Sign in'
        elif 'email the' in row['Attribute']:
            return 'Email'
        elif 'leave a comment for' in row['Attribute']:
            return 'Written Testimony'
        elif 'send a comment to' in row['Attribute']:
            return 'Comment to District Legislator'
        elif 'form email' in row['Attribute']:
            return 'Partner Form Email'
    
    # Default case
    return None

# Apply the function to the DataFrame to create the 'Contact Type' column
normalized_df['Contact Type'] = normalized_df.apply(calculate_contact_type, axis=1)

In [48]:
# Calculate 'Contact' column based on 'Contact Type' and 'Attribute' columns

def calculate_contact(row):
    # Step 1: Check 'Action Verification' and 'Action Completed' values
    if row['Attribute Type'] == 'Action Verification' and row['Action Completed'] == 'Yes':

        # Step 2: Conditions based on 'Contact Type' and 'Attribute' values
        if row['Contact Type'] == 'Sign in':
            return 'Committee'
        elif row['Contact Type'] == 'Written Testimony':
            return 'Committee'
        elif row['Contact Type'] == 'Email':
            if 'committee members' in row['Attribute']:
                return 'Committee'
            elif 'legislators' in row['Attribute']:
                return 'Specific Legislators'
            elif 'members of the House' in row['Attribute']:
                return 'House Democrats'
            elif 'members of the Senate' in row['Attribute']:
                return 'Senate Democrats'
        elif row['Contact Type'] == 'Comment to District Legislator':
            if 'Senator and two Representatives' in row['Attribute']:
                return 'Senator and Representatives'
            if 'Senator' in row['Attribute']:
                return 'Senator'
            elif 'two Representatives' in row['Attribute']:
                return 'Representatives'
        elif row['Contact Type'] == 'Partner Form Email':
            if 'Senator and two Representatives' in row['Attribute']:
                return 'Senator and Representatives'
            elif 'Senator' in row['Attribute']:
                return 'Senator'
            elif 'two Representatives' in row['Attribute']:
                return 'Representatives'
    
    # Default case
    return None

# Apply the function to the DataFrame to create the 'Contact' column
normalized_df['Contact'] = normalized_df.apply(calculate_contact, axis=1)

In [49]:
# Calculate 'Contact Position' based on 'Attribute' column

def calculate_contact_position (row):
    # Step 1: Check 'Action Verification' and 'Action Completed' values
    if row['Attribute Type'] == 'Action Verification' and row['Action Completed'] == 'Yes':

        # Step 2: Conditions based on 'Attribute' values
        if 'pro' in row['Attribute']:
            return 'Pro'
        elif 'con' in row['Attribute']:
            return 'Con'
        elif 'other' in row['Attribute']:
            return 'Other'
        elif 'support' in row['Attribute']:
            return 'Support'
        elif 'oppose' in row['Attribute']:
            return 'Oppose'
        elif 'modify' in row['Attribute']:
            return 'Other'

    # Default case
    return None

# Apply the function to the DataFrame to create the 'Contact Position' column
normalized_df['Contact Position'] = normalized_df.apply(calculate_contact_position, axis=1)

In [50]:
# Calculate 'Contact Count' column based on 'Contact Type' and 'Attribute' columns
def calculate_contact_count(row):
    # Step 1: Check 'Action Verification' and 'Action Completed' values
    if row['Attribute Type'] == 'Action Verification' and row['Action Completed'] == 'Yes':
        
        # Step 2: Condition when 'Contact Type' is 'Email'
        if row['Contact Type'] == 'Email':
            # Ensure 'Attribute' is a string and extract the first number using regex
            if isinstance(row['Attribute'], str):
                number = pd.to_numeric(pd.Series(row['Attribute']).str.extract(r'(\d+)', expand=False)).iloc[0]
                return number if pd.notna(number) else 0
            return 0

        # Step 3: Other conditions based on 'Contact Type' and 'Attribute' values
        elif row['Contact Type'] == 'Sign in':
            return 1
        elif row['Contact Type'] == 'Written Testimony':
            return 1
        elif row['Contact Type'] == 'Comment to District Legislator':
            if row['Contact'] == 'Senator and Representatives':
                return 3
            elif row['Contact'] == 'Representatives':
                return 2
            elif row['Contact'] == 'Senator':
                return 1
        elif row['Contact Type'] == 'Partner Form Email':
            if row['Contact'] == 'Senator and Representatives':
                return 3
            elif row['Contact'] == 'Representatives':
                return 2
            elif row['Contact'] == 'Senator':
                return 1
    
    # Default case
    return None

# Apply the function to the DataFrame to create the 'Contact Count' column
normalized_df['Contact Count'] = normalized_df.apply(calculate_contact_count, axis=1)


In [51]:
# Calculate 'Bill Number' from the 'Attribute' column

# Combine the HB and SB regex patterns into one
bill_pattern = r'(HB \d+\.\d*|HB \d+|SB \d+\.\d*|SB \d+)\?'

# Function to extract and assign bill numbers
def extract_and_assign_bill_numbers(df):
    # Create a mask for rows where 'Attribute Type' is 'Action Verification'
    action_verif_mask = df['Attribute Type'] == 'Action Verification'

    # Initialize the 'Bill Number' column explicitly as object type (string)
    df['Bill Number'] = pd.Series(np.nan, index=df.index, dtype=object)

    # Extract the bill numbers where the mask is True using a single regex
    df.loc[action_verif_mask, 'Bill Number'] = df.loc[action_verif_mask, 'Attribute'].str.extract(bill_pattern)[0]

    # Forward fill the extracted bill numbers within each 'Submission ID' and 'Action Number' group
    df['Bill Number'] = df.groupby(['Submission ID', 'Action Number'])['Bill Number'].transform('first')

    return df

# Apply the function to the entire dataframe at once
normalized_df = extract_and_assign_bill_numbers(normalized_df)

# Reset the index if needed
normalized_df.reset_index(drop=True, inplace=True)


In [52]:
# Define file path
csv_path = r'D:\Users\peter\PythonProjects\PandasTest\data\Week 05 Wed_normalized.csv'

# Write DataFrame to CSV file
normalized_df.to_csv(csv_path, index=False)
